# Funciones Utilizadas Trabajo Fin de Grado

# Predicción de los Precios de las Viviendas en Madrid Capital

# Grado en Estadística y Empresa - Curso 2023/2024

# Andrés Rubio Lafuente


Este archivo contiene el conjunto de funciones que se utilizan para la realización del Trabajo de Fin de Grado.

Cargamos los paquetes necesarios.




In [ ]:
# Básico
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Métodos de Aprendizaje
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score
from scipy.cluster.hierarchy import dendrogram
from scipy.stats import t

# Tiempo de Ejecucion
import time

# Advertencias
import warnings
warnings.simplefilter("ignore")

1. Funcion que elimina los valores nulos y transforma el tipo de variable a numérico.

In [ ]:
def nulos(datos, x):
  indices = []
  valores = []
  i = 0
  while i < len(datos[x]):

      # Vemos que instancias permiten el cambio de dato a tipo numérico.
      try:
            int(datos[x][i])
            str(datos[x][i])

      # Las instancias que no lo permiten es porque son de tipo texto o nulos.
      except:
            indices.append(i)
            valores.append(datos[x][i])
      i += 1

  # Vemos que instancias son de tipo texto o nulos y que valor toman.
  valores = set(valores)
  print(f"Las filas a eliminar son: {indices}, pues toman valores {valores}.")
  print(f"En total se eliminan {len(indices)} filas del conjunto de datos por la variable {x}.")

  # Eliminamos estas instancias que toman valor tipo texto o nulo.
  datos.drop(indices, axis = 0, inplace = True)
  datos.reset_index(drop = True, inplace = True)

  # Cambiamos el tipo de variable a numérico
  datos[x] = datos[x].astype(int)
  return(datos[x])

2. Función que identifica los valores de tipo texto y los transforma a valores nulos.



In [ ]:
def nulos_texto(datos, x):
  indices = []
  valores = []
  i = 0
  while i < len(datos[x]):

      # Vemos que instancias permiten el cambio de dato a tipo numérico.
      try:
            int(datos[x][i])

      # Las instancias que no lo permiten es porque son de tipo texto.
      # Transformamos los valores de tipo texto a valores faltantes.
      except:
            indices.append(i)
            valores.append(datos[x][i])
            datos.at[i,x] = np.nan
      i += 1

  # Vemos que instancias son de tipo texto y que valor toman.
  valores = set(valores)
  datos[x] = pd.to_numeric(datos[x])
  print(f"Las filas con valores faltantes son : {indices}, pues toman valores {valores}.")
  print(f"En total hay {len(indices)} filas con valores faltantes del conjunto de datos para la variable {x}.")
  print(f"La variable {x} es del tipo: {datos.dtypes[x]}")
  return(datos[x])

3. Función para identificar las instancias con valores faltantes.



In [ ]:
def valores_faltantes(datos):

  # Para cada variable, se extraen las filas con valores faltantes
  ind = []
  for i in datos.columns:
      if datos.isna().sum()[i] > 0:
            ind_inicial = datos[datos.isna()[i] == True].index
      ind.append(ind_inicial)

  # Dado que una fila puede tener varios valores faltantes, entonces puede aparecer múltiples veces.
  # Debemos eliminar los indices que se repitan de forma que un indice solo aparezca 1 vez.
  ind_final = set()
  for i in ind:
      ind_final.update(i.tolist())
  return(ind_final)

4. Funcion para generar diagramas de caja o boxplots.

In [ ]:
def boxplot_num(datos, variable_num, variable_cat, variable_orden):

    # Creamos un diagrama de caja o boxplot para una variable numérica en función de una variable categórica.
    # Es decir, por ejemplo podemos obtener el boxplot del precio en función del distrito.

    plt.figure(figsize = (10, 10))
    sns.set(style = 'darkgrid')
    orden = datos.groupby(variable_cat)[variable_orden].mean().sort_values(ascending = False).index
    sns.boxplot(data = datos, x = variable_cat, y = datos[variable_num], hue = variable_cat, order = orden, hue_order = orden, legend = False)
    plt.xticks(rotation = 90)
    plt.xlabel(variable_cat)
    plt.ylabel(variable_num)
    plt.tight_layout()

5. Función que elimina los valores atipicos que son muy extremos.


In [ ]:
def valores_atipicos(datos, variable_num, variable_cat):

  # Identificamos los datos que corresponden a cada distrito.
  datos_categoria = []
  valores = datos[variable_cat].unique()
  for i in valores:
      datos_valores = datos[datos[variable_cat] == i]

      # Calculamos el rango intercuartilico.
      q1 = datos_valores[variable_num].quantile(0.25)
      q3 = datos_valores[variable_num].quantile(0.75)
      IQR = q3 - q1

      # Definimos los limites inferiores y superiores
      lim_inf = q1 - 3 * IQR
      lim_sup = q3 + 3 * IQR

      # Todos aquellos valores que estén fuera de los límites se eliminan del conjunto de datos.
      datos_filtrados = datos_valores[((datos_valores[variable_num] > lim_inf) & (datos_valores[variable_num] < lim_sup)) | (datos_valores[variable_num].isna() == True)]
      datos_categoria.append(datos_filtrados)
  datos_retorno = pd.concat(datos_categoria)
  return(datos_retorno)

6. Función que nos permite aplicar ciertos métodos de aprendizaje supervisado con búsqueda en rejilla de hiperparámetros.

In [ ]:
def regresion_grid(preproceso, metodo, params, X_train, Y_train, X_test, Y_test):

  # Definimos el metodo que se va a aplicar.
  regresor = Pipeline([('preproceso', preproceso),
                       ('regresor', metodo)])

  # Para la validación interna (ajuste de hiperparámetros) utilizamos validación cruzada con k = 10 folds.
  val_interna = KFold(n_splits = 10, shuffle = True, random_state = 129)

  # Empleamos la búsqueda en rejilla y el r2 como métrica de evaluación del modelo.
  grid_search = GridSearchCV(estimator = regresor, param_grid = params, cv = val_interna, scoring = 'r2')

  # Entrenamos el modelo y calculamos el tiempo de entrenamiento del modelo.
  tiempo_inicio = time.time()
  grid_search.fit(X_train, Y_train)
  tiempo_fin = time.time()
  tiempo_ejecucion = tiempo_fin - tiempo_inicio

  # Vemos cuáles son los mejores valores de los hiperparámetros así como el r2 de entrenamiento y de evaluación.
  mejores_hp = grid_search.best_params_
  r2_train = grid_search.best_score_
  mejor_modelo = grid_search.best_estimator_
  pred = mejor_modelo.predict(X_test)
  r2_test = r2_score(Y_test, pred)

  # Guardamos todos los resultados en un dataframe.
  res = pd.DataFrame([[metodo, mejores_hp, r2_train, r2_test, tiempo_ejecucion]])
  res.columns = ['Metodo AS', 'Mejores Hiperparámetros', 'R2 Train', 'R2 Test', 'Tiempo de Ejecución (Segundos)']
  return(res)

7. Función que nos permite aplicar ciertos métodos de aprendizaje supervisado con búsqueda aleatoria de hiperparámetros.

In [ ]:
def regresion_random(preproceso, metodo, params, X_train, Y_train, X_test, Y_test):

  # Definimos el metodo que se va a aplicar.
  regresor = Pipeline([('preproceso', preproceso),
                       ('regresor', metodo)])

  # Para la validación interna (ajuste de hiperparámetros) utilizamos validación cruzada con k = 10 folds.
  val_interna = KFold(n_splits = 10, shuffle = True, random_state = 129)

  # Empleamos la búsqueda en rejilla y el r2 como métrica de evaluación del modelo.
  random_search = RandomizedSearchCV(regresor, params, cv = val_interna, scoring = 'r2', random_state = 129)

  # Entrenamos el modelo y calculamos el tiempo de entrenamiento del modelo.
  tiempo_inicio = time.time()
  random_search.fit(X_train, Y_train)
  tiempo_fin = time.time()
  tiempo_ejecucion = tiempo_fin - tiempo_inicio

  # Vemos cuáles son los mejores valores de los hiperparámetros así como el r2 de entrenamiento y de evaluación.
  mejores_hp = random_search.best_params_
  r2_train = random_search.best_score_
  mejor_modelo = random_search.best_estimator_
  pred = mejor_modelo.predict(X_test)
  r2_test = r2_score(Y_test, pred)

  # Guardamos todos los resultados en un dataframe.
  res = pd.DataFrame([[metodo, mejores_hp, r2_train, r2_test, tiempo_ejecucion]])
  res.columns = ['Metodo AS', 'Mejores Hiperparámetros', 'R2 Train', 'R2 Test', 'Tiempo de Ejecución (Segundos)']
  return(res)

8. Función para Redes Neuronales para búsqueda de hiperparámetros y entrenamiento del modelo

In [ ]:
def redes_neuronales(preproceso, metodo, params, X_train, Y_train, X_test, Y_test):

  # Definimos el metodo que se va a aplicar.
  regresor = Pipeline([('preproceso', preproceso),
                       ('regresor', metodo)])

  # Empleamos la búsqueda en rejilla y el r2 como métrica de evaluación del modelo.
  random_search = RandomizedSearchCV(regresor, params, scoring = 'r2', random_state = 129)

  # Entrenamos el modelo y calculamos el tiempo de entrenamiento del modelo.
  tiempo_inicio = time.time()
  random_search.fit(X_train, Y_train)
  tiempo_fin = time.time()
  tiempo_ejecucion = tiempo_fin - tiempo_inicio

  # Vemos cuáles son los mejores valores de los hiperparámetros así como el r2 de entrenamiento y de evaluación.
  mejores_hp = random_search.best_params_
  r2_train = random_search.best_score_
  mejor_modelo = random_search.best_estimator_
  pred = mejor_modelo.predict(X_test)
  r2_test = r2_score(Y_test, pred)

  # Guardamos todos los resultados en un dataframe.
  res = pd.DataFrame([[metodo, mejores_hp, r2_train, r2_test, tiempo_ejecucion]])
  res.columns = ['Metodo AS', 'Mejores Hiperparámetros', 'R2 Train', 'R2 Test', 'Tiempo de Ejecución (Segundos)']
  return(res)

9. Función que dibuja un dendrograma.

In [ ]:
def dendrograma(model, **kwargs):
    plt.figure(figsize = (20, 12))
    cont = np.zeros(model.children_.shape[0])
    n = len(model.labels_)
    for i, j in enumerate(model.children_):
        cont1 = 0
        for idx in j:
            if idx < n:
                cont += 1
            else:
                cont1 += cont[idx - n]
        cont[i] = cont1
    linkage_matrix = np.column_stack([model.children_, model.distances_, cont]).astype(float)
    _ = dendrogram(linkage_matrix, **kwargs)

10. Función que calcula el intervalo de confianza para la media basado en la distribución t de Student.

In [ ]:
def intervalo_confianza(n, media, desv, conf_level):

  # Obtenemos la desviación típica (se), el valor del estadístico T-Student y los límites inferior y superior
  se = desv / np.sqrt(n)
  est = t.ppf((1 + conf_level) / 2, n-1)
  inf = media - est * se
  sup = media + est * se

  # Aplicamos los formatos a las variables para que tengan 2 decimales y guardamos los datos en un dataframe
  X_medias_f = media.astype(float).round(2).apply(lambda x: '{:0g}'.format(x))
  inf_f = inf.astype(float).round(2).apply(lambda x: '{:0g}'.format(x))
  sup_f = sup.astype(float).round(2).apply(lambda x: '{:0g}'.format(x))
  pd.options.display.float_format = '{:,.0f}'.format
  ic = pd.DataFrame(pd.concat([X_medias_f, inf_f, sup_f], axis = 1, keys = ['Media', 'IC Inferior', 'IC Superior']))
  return(ic)